# Vehicle routing with Timefold in a Kotlin notebook

In [107]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.5.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.16.1")

## Domain

Create the domain data classes:

### Location

In [108]:
import com.fasterxml.jackson.annotation.JsonFormat

@JsonFormat(shape = JsonFormat.Shape.ARRAY)
data class Location(
    val latitude: Double,
    val longitude: Double)

### Visit

In [109]:
import java.time.Duration

data class Visit(

    val name: String,
    val location: Location,
    val load: Int) {

    override fun toString(): String = name
}

### Vehicle

In [110]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningListVariable


@PlanningEntity
data class Vehicle(

    val name: String,
    val homeLocation: Location,
    val capacity: Int) {

    @PlanningListVariable
    var visits: MutableList<Visit> = ArrayList()
    
    // No-arg constructor required for Timefold
    constructor() : this("", Location(0.0, 0.0), 0)

    override fun toString(): String = name
}

## Constraints

In [111]:
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.Joiners
import java.time.Duration

class VehicleRoutingConstraintProvider : ConstraintProvider {

    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint>? {
        return arrayOf(
            // Hard constraints
            capacity(constraintFactory)
            // Soft constraints
        )
    }
    
    fun capacity(constraintFactory: ConstraintFactory): Constraint {
        // TODO Not the most efficient implementation
        return constraintFactory
            .forEach(Vehicle::class.java)
            .expand({ vehicle -> vehicle.visits.sumOf { it.load } })
            .filter({ vehicle, load -> load > vehicle.capacity })
            .penalize(HardSoftScore.ONE_HARD, { vehicle, load -> load - vehicle.capacity })
            .asConstraint("vehicle-routing", "Capcity");
    }

}

## Schedule

In [112]:
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import kotlinx.serialization.Transient


@PlanningSolution
data class Schedule(
    
    val name: String,
    @PlanningEntityCollectionProperty
    val vehicles: List<Vehicle>,
    @ProblemFactCollectionProperty
    @ValueRangeProvider
    val visits: List<Visit>) {

    @PlanningScore
    @Transient
    var score: HardSoftScore? = null

    // No-arg constructor required for Timefold
    constructor() : this("", emptyList(), emptyList())

}

## Read input data

In [113]:
import java.io.File
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue

val mapper = jacksonObjectMapper()
val problem: Schedule =  mapper.readValue(File("vehicle-routing-data.json"))

## Solve

In [114]:
import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver

val solverFactory: SolverFactory<Schedule> = SolverFactory.create(SolverConfig()
        .withSolutionClass(Schedule::class.java)
        .withEntityClasses(Vehicle::class.java)
        .withConstraintProviderClass(VehicleRoutingConstraintProvider::class.java)
        // The solver runs only for 5 seconds on this small dataset.
        // It's recommended to run for at least 5 minutes ("5m") otherwise.
        .withTerminationSpentLimit(Duration.ofSeconds(5)))

println("Solving the problem ...")
val solver: Solver<Schedule> = solverFactory.buildSolver()
val solution: Schedule = solver.solve(problem)
println("Solving finished with score (${solution.score}).")

Solving the problem ...
Solving finished with score (0hard/0soft).


## Visualize the schedule

In [115]:
HTML(buildString {
    append("<p style='font-size: x-large'>Score: ${solution.score}</p>")
    append("<ul>")
    for (vehicle in solution.vehicles) {
        append("<li>${vehicle.name}: ${vehicle.visits.joinToString(", ")}</li>")
    }
    append("</ul>")
})

Score: 0hard/0soft Vehicle A: NIEUWKERKEN-WAAS, LISSEWEGE, KOMEN, BULSKAMP, BOURLERS, BERSILLIES-L'ABBAYE, BAVIKHOVE, AVE-ET-AUFFE, ANTHISNES Vehicle B: MONT_NAM., EVELETTE, DAMME, CHIEVRES, BLERET Vehicle C: LES_BONS_VILLERS, GONDREGNIES, FONTAINE-L'EVEQUE, ELVERDINGE, DONSTIENNES Vehicle D: MAULDE, HAREN_BRUSSEL, GUIGNIES, GELINDEN Vehicle E: JURBISE, HUMBEEK, HOFSTADE_BT., HEPPIGNIES Vehicle F: NAAST, MERKSPLAS, MAFFLE, LANDEGEM Vehicle G: SAINT-MARTIN, ROSMEER, RENINGE, POUPEHAN, PEPINGEN, ORET, OLLIGNIES Vehicle H: VAUX-LEZ-ROSIERES, TAILLES, SOMZEE, SINT-KWINTENS-LENNIK, SELANGE Vehicle I: WETTEREN, VOSSEM, VILLERS-L'EVEQUE, TILFF Vehicle J: XHENDELESSE, WARNANT-DREYE

## Analyze the score

Let's break down the score per constraint:

In [116]:
import ai.timefold.solver.core.api.solver.SolutionManager

val solutionManager = SolutionManager.create(solverFactory)
val scoreAnalysis = solutionManager.analyze(solution)

And visualize it:

In [117]:
HTML(buildString {
    append("<p style='font-size: x-large'>Score: ${scoreAnalysis.score}</p>")
    append("<ul>")
    for (constraint in scoreAnalysis.constraintMap().values) {
        append("<li>${constraint.constraintRef().constraintName}: ${constraint.score.toShortString()}</li>")
    }
    append("</ul>")
})

Score: 0hard/0soft Capcity: 0

## Statistics

For a big dataset, a schedule visualization is often too verbose.
Let's visualize the solution through statistics:

In [118]:
%use kandy

### TODO

## Conclusion

To learn more about planning optimization, visit [timefold.ai](https://timefold.ai).